In [ ]:
# 데이터 로드
from google.colab import drive

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from keras.optimizers import Adam

# 정규화 패키지
from sklearn.preprocessing import MinMaxScaler

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### 함수 정의 구간
# 데이터 업로드 _utf
def upload_data_utf(file_name):
  rtn = pd.read_csv(file_name, encoding = 'utf-8')
  rtn = pd.DataFrame(rtn)

  return rtn

def upload_data_cp(file_name):
  rtn = pd.read_csv(file_name, encoding = 'cp949', delimiter = '\t')
  rtn = pd.DataFrame(rtn)

  return rtn

### 데이터 정규화
  # Min-Max Scaling
  # 0에서 1사이로 정규화
def scaling(data, loctaion):

  # 스케일 대상 column
  scaleColumn = ['건축년도', '전용면적'] + location
  rtn = data

  # 데이터 정규화
  scaler = MinMaxScaler()
  rtn[scaleColumn] = scaler.fit_transform(rtn[scaleColumn])

  return rtn

In [ ]:
# XDATA 파일 업로드
x_data1 = upload_data_utf("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/SuwonData/XDATA(Special).csv")
x_data2 = upload_data_utf("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/AnyangData/XDATA(Special).csv")

x_data = pd.concat([x_data1, x_data2], ignore_index=True)

y_data1 = x_data1[['거래금액', '거래일자']]
y_data2 = x_data2[['거래금액', '거래일자']]

In [ ]:
# 계절지수 데이터 업로드
jisoo = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/계절지수.xlsx")

In [ ]:
# 날짜 타입으로 바꾸기
y_data1['거래일자'] = pd.to_datetime(y_data1['거래일자'])
y_data2['거래일자'] = pd.to_datetime(y_data2['거래일자'])
jisoo['일자'] = pd.to_datetime(jisoo['일자'])

# 년월만 있는 '년월' 열 추가
jisoo['년월'] = jisoo['일자'].dt.to_period('M')
y_data1['년월'] = y_data1['거래일자'].dt.to_period('M')
y_data2['년월'] = y_data2['거래일자'].dt.to_period('M')

<ipython-input-7-d8fe2808d6c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data1['거래일자'] = pd.to_datetime(y_data1['거래일자'])
<ipython-input-7-d8fe2808d6c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data2['거래일자'] = pd.to_datetime(y_data2['거래일자'])
<ipython-input-7-d8fe2808d6c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [ ]:
# 같은 년월의 가격 지수로 거래금액 나누기
for i in range(len(y_data1)):
  for j in range(len(jisoo)):
    if y_data1['년월'][i] == jisoo['년월'][j]:
       y_data1['거래금액'][i] = float(y_data1['거래금액'][i].replace(',', '')) / jisoo['경부2권'][j] * 100

# 같은 년월의 가격 지수로 거래금액 나누기
for i in range(len(y_data2)):
  for j in range(len(jisoo)):
    if y_data2['년월'][i] == jisoo['년월'][j]:
       y_data2['거래금액'][i] = float(y_data2['거래금액'][i].replace(',', '')) / jisoo['경부1권'][j] * 100

<ipython-input-8-d48053712487>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data1['거래금액'][i] = float(y_data1['거래금액'][i].replace(',', '')) / jisoo['경부2권'][j] * 100
<ipython-input-8-d48053712487>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data2['거래금액'][i] = float(y_data2['거래금액'][i].replace(',', '')) / jisoo['경부1권'][j] * 100


In [ ]:
# 입지 정보 칼럼명
location = ['위도','경도',
            '약국', '의원','병원', '대병원',
            '대형상권','공원', '도서관',
            '유치원', '초등학교', '중학교', '고등학교', '학원',
            '지하철',
            '약국 수', '의원 수', '병원 수', '대병원 수',
            '대형상권 수','공원 수','도서관 수',
            '유치원 수', '초등학교 수', '중학교 수', '고등학교 수', '학원 수',
            '지하철 수']

y_data = pd.concat([y_data1, y_data2], ignore_index=True)
y_data = y_data[['거래금액']]

x_data = x_data[['건축년도', '전용면적'] + location]

In [ ]:
# 텐서플로 형식에 맞게 32bit로 변경
x_data = x_data.astype('float32')
y_data = y_data.astype('float32')

In [ ]:
# 데이터 정규화
x_data = scaling(x_data, location)
x_data = x_data[['건축년도', '전용면적'] + location]

In [ ]:
# 훈련 세트와 테스트 세트로 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
# 다층 퍼셉트론 회귀 모델 생성
mlp_regressor = MLPRegressor(random_state=42)

# 튜닝할 매개변수 분포 정의
param_dist = {
    'hidden_layer_sizes': [(60,), (120, 60,), (240, 120, 60,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'alpha': uniform(0.0001, 0.01),
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': randint(100, 300)
}

# Randomized Search 수행
random_search = RandomizedSearchCV(estimator=mlp_regressor,
                                   param_distributions=param_dist,
                                   n_iter=10, cv=5, scoring='neg_mean_squared_error',
                                   random_state=42)

random_search.fit(X_train, y_train)

# 최적의 매개변수 출력
print("Best Parameters:", random_search.best_params_)

In [ ]:
# 다층 퍼셉트론 회귀 모델 생성
mlp_regressor = MLPRegressor(random_state=42)

# 튜닝할 매개변수 분포 정의
param_dist = {
    'hidden_layer_sizes': [(60,), (120, 60,), (240, 120, 60,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'alpha': uniform(0.0001, 0.01),
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': randint(100, 300)
}

In [ ]:
# Randomized Search 수행
random_search = RandomizedSearchCV(estimator=mlp_regressor, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (171) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (171) reached and the optimization hasn't conver

RandomizedSearchCV(cv=5, estimator=MLPRegressor(random_state=42),
                   param_distributions={'activation': ['relu', 'tanh',
                                                       'logistic'],
                                        'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7edd6de27c70>,
                                        'hidden_layer_sizes': [(60,), (120, 60),
                                                               (240, 120, 60)],
                                        'learning_rate': ['constant',
                                                          'invscaling',
                                                          'adaptive'],
                                        'max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7eddf359a8f0>},
                   random_state=42, scoring='neg_mean_squared_error')

In [ ]:
# 최적의 매개변수 출력
print("Best Parameters:", random_search.best_params_)

Best Parameters: {'activation': 'relu', 'alpha': 0.005347746602583892, 'hidden_layer_sizes': (240, 120, 60), 'learning_rate': 'invscaling', 'max_iter': 287}


In [ ]:
# 최적의 모델 추출
best_mlp_model = random_search.best_estimator_

# 테스트 데이터로 예측
y_pred = best_mlp_model.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 54637716.0


In [ ]:
# R-squared 계산
r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

R-squared: 0.9264749516863847


In [ ]:
# 모델 초기화
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(240, activation='relu', input_shape=(30,)))
model.add(tf.keras.layers.Dense(120, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

# 모델 컴파일
optimizer = Adam(learning_rate=0.005347746602583892)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# 모델 훈련
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# 모델 훈련
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
334/334 [==============================] - 2s 4ms/step - loss: 1050607936.0000 - val_loss: 355637344.0000
Epoch 2/100
334/334 [==============================] - 1s 3ms/step - loss: 341224864.0000 - val_loss: 217892336.0000
Epoch 3/100
334/334 [==============================] - 1s 3ms/step - loss: 270415648.0000 - val_loss: 170251488.0000
Epoch 4/100
334/334 [==============================] - 1s 3ms/step - loss: 240633792.0000 - val_loss: 179094192.0000
Epoch 5/100
334/334 [==============================] - 1s 3ms/step - loss: 235493664.0000 - val_loss: 170185696.0000
Epoch 6/100
334/334 [==============================] - 1s 3ms/step - loss: 216652992.0000 - val_loss: 135846704.0000
Epoch 7/100
334/334 [==============================] - 1s 3ms/step - loss: 215176512.0000 - val_loss: 139971600.0000
Epoch 8/100
334/334 [==============================] - 1s 3ms/step - loss: 213152256.0000 - val_loss: 127050376.0000
Epoch 9/100
334/334 [==============================] - 1s 4ms/s

In [ ]:
# 모델 평가
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

84/84 [==============================] - 0s 2ms/step - loss: 52976000.0000
Mean Squared Error on Test Set: 52976000.0


In [ ]:
# 데이터에 대한 예측
predictions = model.predict(X_test)
print(f'Predictions for new data: {predictions}')

84/84 [==============================] - 2s 8ms/step
Predictions for new data: [[36062.38 ]
 [39616.19 ]
 [40432.395]
 ...
 [47163.758]
 [57660.14 ]
 [61678.254]]


In [ ]:
# R-squared 계산
r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

R-squared: 0.9287111100010249


In [ ]:
# rmse 함수
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

# adjusted r2 함수
def calculate_adjusted_r2(y_true, y_pred, num_features):
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - num_features - 1)
    return adjusted_r2

In [ ]:
# rmse 계산
rmse_value = calculate_rmse(y_test, predictions)
print(f"RMSE: {rmse_value}")

# adjusted R-squared 계산
adjusted_r2_value = calculate_adjusted_r2(y_test, predictions, 30)
print(f"Adjusted R-squared: {adjusted_r2_value}")

RMSE: 7278.46044921875
Adjusted R-squared: 0.9279016222266232
